# Chapter 21 - Working with Categorical Variables

### Steve Elston

Up to now, we have only worked with **numeric variables**. However, **categorical variables** are extremely common. In fact, most real-world data science problems involve one or more categorical variables. Indeed, some types of analysis involve only categorical variables. For examine, in natural language processing, all variables are categorical in the form of words or phrases. 

Machine learning models can have categorical variables as features. But, most models require numeric predictor variables. The question is, how can we transform categorical variables into numeric variables?  

In this notebook you will be introduced to working with categorical variables as features in regression models. The methods explored apply to most any machine learning model.  

## Preparing the Data Set

In this section we will prepare the data set we will use in this notebook. As a first step, execute the code in the cell below to import the required packages.

In [3]:
import pandas as pd
import numpy as np
import numpy.random as nr
import statsmodels.api as sm
import statsmodels.formula.api as smf  
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from patsy import dmatrices

%matplotlib inline

Execute the cell below to perform load the dataset into a data frame and examine the data types of the columns. 

In [4]:
auto_data = pd.read_csv('../data/AutoPricesClean.csv')
auto_data.dtypes                      

Unnamed: 0             int64
symboling              int64
normalized_losses     object
make                  object
fuel_type             object
aspiration            object
num_of_doors          object
body_style            object
drive_wheels          object
engine_location       object
wheel_base           float64
length               float64
width                float64
height               float64
curb_weight            int64
engine_type           object
num_of_cylinders      object
engine_size            int64
fuel_system           object
bore                 float64
stroke               float64
compression_ratio    float64
horsepower             int64
peak_rpm               int64
city_mpg               int64
highway_mpg            int64
price                  int64
dtype: object

There are a number of numeric variables and categorical variables, of type `object`.

Next, execute the code in the cell below to do the following:   

- Remove the first 3 columns of the data frame. 
- Normalize the numeric columns of the data frame, except for the last 3 we can use as labels.  
- Examine the head of the data frame.   

In [ ]:
## Get a list of columns that are not of type object   
auto_data.drop(auto_data.columns[:3], axis=1, inplace=True)
numeric_columns = [col for col_type,col in zip(auto_data.iloc[:,:-3].dtypes,auto_data.iloc[:,:-3].columns) if col_type in ['int64','float64']]
auto_data.loc[:,numeric_columns] = normalize(auto_data.loc[:,numeric_columns])
auto_data.head()

## Categorical Variables and the Model Matrix

Most machine learning models can only work with numeric variables. Therefore, ee need to encode the categorical variables into one or more numeric variables. The common approach is to convert the categorical variable to a set of binary **dummy variables** or **indicator variables**. We call this process **one-hot encoding** since only one of the dummy variables will be encoded as a 1 for each category or level of the variable. 

We will work with the Python [patsy](https://patsy.readthedocs.io/en/latest/quickstart.html) package which creates **design matrices** from Pandas data frames. Patsy creates the design matrices using the, by now familiar, R-style modeling [formulas](https://patsy.readthedocs.io/en/latest/formulas.html). These design matrices can be used directly in statsmodels or scikit-learn models. 

> **Note:** In this lesson we will use the term design matrices. However, you often see the term **model matrices**. Don't be confused! These terms mean the same thing.

### Design matrix with an intercept term

To start our exploration of how model matrices are constructed we will start with an example using only numeric variables. We will use the [dmatrices](https://patsy.readthedocs.io/en/latest/API-reference.html) function from the patsy package, which returns both the feature and label matrices. Execute the code in the cell below to construct the label and design matrices for the model formula shown and examine the results. 

In [ ]:
Y, X = dmatrices('city_mpg ~ body_style', data=auto_data)
print('Head of label matrix \n{}\n'.format(Y[:5]))
print('Design matrix \n{}'.format(X[:5]))

The label array, $Y$, looks much as you might expect. A one dimensional array of label values.  

The columns of the feature matrix might surprise you.  The first column is all 1's. This is the feature column for the **intercept term**. There are 4 containing dummy variables encoding the body style.

But notice that first two rows have only 0s for the dummy variables? How can this be? It turns out the intercept term is the mean of the first category of `body_style`. The other columns represent **contrasts** between the intercept and the **effect size** of the other categories. 

> **Exercise 21-1:** To better understand how the encoding of the dummy variables as contrasts works do the following:       
> 1. Compute an ols model of `city_mpg` by `body_style` using the `ols` function from `statsmodels.formula.api`.    
> 2. Print the summary of this model.   
> 3. Answer the questions below.   

In [ ]:
## Code goes here


> 1. Notice that there are 4 dummy variables for the 5 possible body styles. Which body style is represented by the intercept term?     
> 2. Which of the 4 contrast terms do you consider to be significant? What does this contrast tell you about the average city MPG for this category of auto vs. the category represented by the intercept term?    
> 3. Based on adjusted $R^2$ and F-statistic does this model explain any of the variance of the city MPG?  
> **End of exercise.**

###  Design matrices without an intercept term 

We may not always want an intercept term. Patsy allows us to build a design matrix without an intercept term by including -1 in the formula. Execute the code in the cell below which does just this.     

In [ ]:
Y, X = dmatrices('city_mpg ~ -1 + body_style', data=auto_data)
print('Head of label matrix \n{}\n'.format(Y[:5]))
print('Design matrix \n{}'.format(X[:5]))

The result is an array with 5 dummy variables, representing the 5 levels of the categorical variable. There is no longer an intercept column. 

Each of the levels of the categorical variable are now represented by a dummy variable. These dummy variables no represent the means for each level, not the contrasts. The feature matrix no longer has a column of 1s. By not using contrasts, the need for an intercept term is eliminated. 

We can test that this design matrix is **orthogonal**. By orthogonal, we mean that each dummy variable is not linearly dependent on any other. An orthogonal design matrix has several advantages. First, the inverse of the covariance matrix is guaranteed to exist. Second, we can directly interpret the model coefficients in terms of the response to the values of the predictor variables.  

The code in the cell below demonstrates the orthogonality of the design matrix by taking the dot products of every pairwise combination. Recall that the dot product between orthogonal vectors is zero. Execute this code and examine the results. 

In [ ]:
from itertools import combinations
for i,j in combinations(range(5), 2):
    print('For columns {0:1d} and {1:1d} the dot product = {2:3.2f}'.format(i,j,np.dot(X[:,i],X[:,j])))

>**Exercise 21-2:** You will now investigate how dropping the intercept term changes the behavior and interpretation of the model coefficients by doing the following:
     
> 1. Compute an ols model of `city_mpg` by `body_style`, with no intercept term, using the `ols` function from `statsmodels.formula.api`.    
> 2. Print the summary of this model.   
> 3. Answer the questions below.   

In [ ]:
## Your code goes here


> Notice that the 5 model coefficients represent the mean for each category of auto and are all now significant. But, that does not mean the coefficient values tells you anything useful. 
> 1. Consider the confidence intervals of the coefficients. Are there their any pairs of coefficients that do not have overlapping confidence intervals? 
> 2. Can you conclude that differences in body style reliably predict the effect size of changing body style, or not?   
> ** End of exercise.**

## Design Matrices with Mixed Type Variables

It is also possible to create design matrices with both categorical variables and continuous numeric variables. The numeric variables must be normalized to be on the same scale as the dummy variables.      

The resulting design matrix, uses the means of the levels of a categorical variable as the model coefficients. In other words, these coefficients represent the value for each level rather than contrasts with an intercept.    

Execute the code in the cell below to create a design matrix with no intercept term with categorical variable `body_style` and normalized continuous variables `engine_size` and `curb_weight`, with no intercept. 

In [ ]:
Y, X = dmatrices('city_mpg ~ -1 + body_style + engine_size + curb_weight', data=auto_data)
print('Head of label matrix \n{}\n'.format(Y[:5]))
print('Design matrix \n{}'.format(np.round(X[:5], 4)))

There are 7 columns in this design matrix. The first 5 columns are the dummy variables for the categorical variables for the contrasts of the levels. The last two columns are the values of the normalized numeric columns.   

One question you could ask, is if the columns of the design matrix are orthogonal? Execute the code in the cell below to find out.     

In [ ]:
for i,j in combinations(range(7), 2):
    print('For columns {0:1d} and {1:1d} the dot product = {2:3.2f}'.format(i,j,np.dot(X[:,i],X[:,j])))

Examine the results above. The dummy variables are orthogonal as they should be. However, the dummy variables are not orthogonal to the numeric variables. Further, the numeric columns are not orthogonal.          

> **Exercise 21-3:** You will now compute and evaluate an OLS model using the categorical variable and numeric variables.    
> 1. Compute an ols model of `city_mpg` by `body_style`, `engine_size` and `curb_weight`, using the `ols` function from `statsmodels.formula.api`.    
> 2. Print the summary of this model.   
> 3. Answer the questions below.   

In [ ]:
## Your code goes here.  


> Examine these results.   
> 1. Are the coefficients of the intercept and the numeric variables significant?    
> 2. Do any of the dummy variables have significance? Given the significance, or lack thereof, do you think these contrasts are predictive?    
> 3. Based on the $R^2$ and F-statistic does this model explain more of the variance of city MPG than the models you computed for Exercises 21-1 and 21-2. 
> **End of exercise.**

> **Exercise 19-4:** You will now create and evaluate an OLS model using means of the levels rather than contrasts.       
> 1. Compute an ols model of `city_mpg` by `body_style`, `curb_weight` and `engine_size`, with no intercept term, using the `ols` function from `statsmodels.formula.api`.    
> 2. Print the summary of this model.   
> 3. Answer the questions below.   

In [ ]:
## Your code goes here

> 1. Do the confidence intervals of the coefficients of the dummy variables indicate that there are any significant differences between them? 
> 2. Compare the $R^2$ and F-statistic to the model you computed in Exercise 21-3. Does this model do any better at explaining the variance of the city MPG?
> **End of exercise.**

#### Copyright 2017, 2018, 2019, 2020, 2021 Stephen F Elston. All rights reserved. 